## Importing Libraries

In [1]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import os
import pandas as pd
import re
import numpy as np

## Parsing XML files
This function would parse XML file and would extract important information from xml tag 'body' 
    and returns list containing dict whcih would have text

In [2]:
fields = ['Id', 'Text', 'Topic']

def parseXML(xmlfile, start_count): 
    #create element tree object 
    print("File", xmlfile)
    tree = ET.parse(xmlfile) # Creates a parse tree 
    topic = xmlfile.split("XML-Files")[1].split("_")[0] # Getting xml file_name from string containing directory/file_name.xml 
    # get root element using which we would parse xml tree
    root = tree.getroot() 
    # create empty list for news items 
    newsitems = [] 
    count = start_count
    # iterate news items 
    for each_row in root.iter("row"):#  iterates only over those elements that have the required tag:"row" 
        news = {}
        news["Id"] = count
        news["Text"] = each_row.attrib["Body"]
        news["Topic"] = topic
        count=count+1
        newsitems.append(news) #[{id:1,text:" Some text  ",Topic : "AI"},{{id:1,text:" Some text  ",Topic : "AI"},...........]
    # return news items list 
    print("len", len(newsitems))
    return newsitems # To save this dict on csv we are putting dict into a list 

## Saving in Csv files
Writes the list of dict returned from above function onto a csv file 

In [3]:
def savetoCSV(newsitems, filename): 
    # writing to csv file 
    with open(filename, 'w', encoding = 'utf-8') as csvfile: 
        # creating a csv dict writer object 
        writer = csv.DictWriter(csvfile, fieldnames = fields) 
        # writing headers (field names) 
        writer.writeheader() 
        # writing data rows 
        writer.writerows(newsitems)

In [4]:
def filterpostfiles(filename):
    """Returns the name of  xml files present in the directory CML Files """
    return filename.endswith("Posts.xml") 
postfiles = filter(filterpostfiles, os.listdir("XML-Files"))

# specifying the fields for csv file 
fields = ['Id', 'Text', 'Topic']  
start_count = 0
for each_file in postfiles:
    print(each_file)
    # parse xml file 
    newsitems = parseXML("XML-Files/"+each_file, start_count)
    #Get only name of xml file by sepearting out extension use the same name to save file as .csv 
    csv_filename = each_file.split('.')[0] + ".csv"
    print("csv_filename", csv_filename)
    # store news items in a csv file 
    savetoCSV(newsitems, "CSV-Files/" + csv_filename)
    start_count = len(newsitems) + start_count

AImeta_Posts.xml
File XML-Files/AImeta_Posts.xml
len 654
csv_filename AImeta_Posts.csv
AI_Posts.xml
File XML-Files/AI_Posts.xml
len 16760
csv_filename AI_Posts.csv
ComputerGraphicsMeta_Posts.xml
File XML-Files/ComputerGraphicsMeta_Posts.xml
len 299
csv_filename ComputerGraphicsMeta_Posts.csv
ComputerGraphics_Posts.xml
File XML-Files/ComputerGraphics_Posts.xml
len 6070
csv_filename ComputerGraphics_Posts.csv
CSMeta_Posts.xml
File XML-Files/CSMeta_Posts.xml
len 1585
csv_filename CSMeta_Posts.csv
CS_Posts.xml
File XML-Files/CS_Posts.xml
len 81429
csv_filename CS_Posts.csv
DataScienceMeta_Posts.xml
File XML-Files/DataScienceMeta_Posts.xml
len 509
csv_filename DataScienceMeta_Posts.csv
DataScience_Posts.xml
File XML-Files/DataScience_Posts.xml
len 54869
csv_filename DataScience_Posts.csv


In [6]:
def filtercsvpostfiles(filename):
    """ Returns all the csv file name extracted from XML which are stored in CSV files"""
    return filename.endswith("Posts.csv") 
csvpostfiles = filter(filtercsvpostfiles, os.listdir("CSV-Files"))

## Merging All the csv files into one single file

In [7]:
total_dataframe = pd.DataFrame(columns=fields)
for each_file in csvpostfiles:
    print("each file", each_file)
    df = pd.read_csv("CSV-Files/"+ each_file)
    print("shape", df.shape)
    total_dataframe = total_dataframe.append(df, ignore_index=True)

each file AImeta_Posts.csv
shape (654, 3)
each file AI_Posts.csv
shape (16760, 3)
each file ComputerGraphicsMeta_Posts.csv
shape (299, 3)
each file ComputerGraphics_Posts.csv
shape (6070, 3)
each file CSMeta_Posts.csv
shape (1585, 3)
each file CS_Posts.csv
shape (81429, 3)
each file DataScienceMeta_Posts.csv
shape (509, 3)
each file DataScience_Posts.csv
shape (54869, 3)


In [8]:
print("Shape of total dataframe", total_dataframe.shape)

Shape of total dataframe (162175, 3)


## Removing Null values

In [9]:
total_dataframe = total_dataframe[~total_dataframe['Text'].isna()]
total_dataframe = total_dataframe[~total_dataframe['Text'].isnull()]
print("Shape of total dataframe after null value remove", total_dataframe.shape)

Shape of total dataframe after null value remove (161423, 3)


## Saved the required form of data in QuestionsDataSet

In [9]:

total_dataframe.to_pickle("QuestionsDataSet.pkl")